# <span style="font-family:Courier New; color:#CCCCCC">**Named Entity Recognition CRF**</span>

In [20]:
import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002
conll2002.iob_sents('esp.train') # Train, ned.train => Neerlandès
conll2002.iob_sents('esp.testa') # Dev
data = conll2002.iob_sents('esp.testb') # Test

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\jerez\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [39]:
from NER_evaluation import *
entities = collect_ne(data[:5])
other = entities.copy()
other[4] = []
eval_metrics, eval_agg_ent = compute_metrics(entities, other)
eval_agg_ent

{'ORG': {'correct': 2,
  'incorrect': 0,
  'partial': 0,
  'missed': 1,
  'spurious': 0,
  'possible': 3,
  'actual': 2},
 'PER': {'correct': 2,
  'incorrect': 0,
  'partial': 0,
  'missed': 0,
  'spurious': 0,
  'possible': 2,
  'actual': 2},
 'LOC': {'correct': 2,
  'incorrect': 0,
  'partial': 0,
  'missed': 0,
  'spurious': 0,
  'possible': 2,
  'actual': 2},
 'MISC': {'correct': 3,
  'incorrect': 0,
  'partial': 0,
  'missed': 2,
  'spurious': 0,
  'possible': 5,
  'actual': 3}}

In [2]:
import nltk
nltk.download('treebank')
train = nltk.corpus.treebank.tagged_sents()[:3000]
test = nltk.corpus.treebank.tagged_sents()[3000:]

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Jordi\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [3]:
from feature_getter import Feature_getter as fg

In [4]:
model = nltk.tag.CRFTagger(fg())

In [5]:
model.train(train, 'model.crf.tagger')

KeyError: "[E001] No component 'tagger' found in pipeline. Available names: ['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner']"